## Step 1: Create $100M portfolio and upload positions to GS Financial Cloud

In [ ]:
import datetime as dt
import warnings
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from IPython.display import display

from gs_quant.markets.portfolio import Portfolio
from gs_quant.markets.portfolio_manager import PortfolioManager
from gs_quant.markets.position_set import PositionSet
from gs_quant.markets.report import FactorRiskReport
from gs_quant.session import GsSession
from gs_quant.target.reports import PositionSourceType, ReportType
from mpl_toolkits.mplot3d import Axes3D
warnings.filterwarnings("ignore")

GsSession.use()


portfolio = Portfolio(name='My New Portfolio')
portfolio.save()
print(f"'{portfolio.name}' Portfolio successfully created.")

position_set = PositionSet.from_list(
    positions=['ARKK'],
    date=dt.date(day=1, month=8, year=2022)
)

position_set.reference_notional = 1000000
position_set.resolve()

pm = PortfolioManager(portfolio.id)
pm.update_positions([position_set])
print(f"Uploaded positions to '{portfolio.name}'.")


## Step 2: Configure risk engines to run three different factor models

In [ ]:
trading_model_ids = ['AXIOMA_AXUS5T', 'BARRA_USFAST', 'WOLFE_QES_US_AC_2']

for model_id in trading_model_ids:
    risk_report = FactorRiskReport(
        report_type=ReportType.Portfolio_Factor_Risk,
        position_source_type=PositionSourceType.Portfolio,
        risk_model_id=model_id,
        position_source_id=portfolio.id
    )
    risk_report.save()
    print(f'Calculating risk with {model_id}...')

pm.schedule_reports()

## Step 3 : Get factor risk and attribution results from GS Financial Cloud

In [ ]:
pm = PortfolioManager('MP1B4W92HSFZRTKT') # replace with portfolio for the UI part of the demo
risk_report = pm.get_factor_risk_report('BARRA_USFAST')
attr_data = risk_report.get_factor_exposure(
    start_date=risk_report.earliest_start_date,
    end_date=risk_report.latest_end_date,
    factor_names=['Beta', 'Value', 'Liquidity', 'Prospect', 'Downside Risk', 'Short Interest',
                  'Size', 'Momentum', 'Sentiment', 'Growth', 'Seasonality', 'Long-Term Reversal']
)
for col in attr_data.columns:
    if col != 'Date':
        attr_data[col] = attr_data[col].div(1000000).round(2)

pd.options.display.float_format = '$ {:,.2f} MM'.format

display(attr_data)

In [ ]:
fig = plt.figure(figsize=(30, 20))
ax = fig.add_subplot(projection='3d')
colors = sns.color_palette("Spectral", 12)

attr_data = attr_data[[col for col in attr_data.columns.tolist() if col not in ['Date', 'Specific', 'Market', 'Industry', 'Style', 'Total', 'Factor']]]
attr_data = attr_data.sort_values(by=0, axis=1, ascending=False)

for index, row in attr_data.iterrows():
    ax.bar(attr_data.columns.tolist(), row.to_numpy(), zs=attr_data.shape[0]-index, zdir='y', color=colors, alpha=1, width=1)

ax.set_xticklabels(attr_data.columns.tolist(), rotation=47, ha='right')
plt.yticks(np.arange(0, attr_data.shape[0], attr_data.shape[0]/12))
ax.set_yticklabels(['Jan 2023', 'Dec', 'Nov', 'Oct', 'Sept', 'Aug', 'Jul', 'Jun', 'May', 'Apr', 'Mar', 'Feb 2022'], rotation=-24, ha='left')
ax.set_zlabel('Exposure (Millions)')

plt.show()

